# Impyte Documentation
This is a first practical attempt to clarify the usage of `impyte`. It's a collection of easy applicable and reproducible examples that you could use to simplyify your data processing workflow.

## Importing and generating sample data
In order to show some of the features of the library, we'll be using Kaggle's HR data that can be found [here](https://www.kaggle.com/ludobenistant/hr-analytics).

In [1]:
import sys
sys.path.append('..')

In [2]:
# import library and data set
from importlib import reload
import impyte
reload(impyte)
import pandas as pd
from tools.testing_sets import TestingSetCreator
from tools.data_prep import remove_random

In [3]:
#from impyte import impyte

In [4]:
data = pd.read_csv('data/hr_test.csv')

In [5]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


## Add random missing values
In order to show some of the pattern visualization and imputation methods, we need incomplete data sets. To achieve this, we'll be using a helper function that deletes values at random. 25 % of all values from each category will be deleted.

In [6]:
data = remove_random(data, .25, randomize_index=False)

In [7]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.75,3.0,260.0,2.0,0.0,0.0,0.0,technical,low
1,NaN,0.70,5.0,264.0,2.0,0.0,0.0,0.0,technical,low
2,NaN,0.50,3.0,231.0,3.0,0.0,0.0,0.0,sales,low
3,NaN,0.79,4.0,272.0,3.0,0.0,0.0,0.0,technical,low
4,NaN,0.90,3.0,138.0,3.0,0.0,0.0,0.0,marketing,medium


In [8]:
data.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000
mean,0.612899,0.716248,3.803487,201.121094,3.497778,0.144547,0.238359,0.021197
std,0.248637,0.171257,1.231349,49.985004,1.458961,0.351656,0.426094,0.144044
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [9]:
len(data)

14999

## Testing rudimentary features
Below is a first trial run of some of the features implemented by impyter and its helper classes.

## `NanChecker`
Functionality testing of `NanChecker` class.

In [10]:
nan_checker = impyte.NanChecker()

#### `NanChecker.is_nan(data, nan_vals=None, recursive=True)`
Detect missing values (NaN in numeric arrays, empty strings in string arrays). NaN detection feature also enables NaN-value ingestion as parameter values.

In [11]:
# [True, False, False]
print(nan_checker.is_nan(["", 'None', 'NaN']))

# [True, True, True]
print(nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN']))

[True, False, False]
[True, True, True]


In [12]:
# Recursive nan detection
# [True, True, False, [False, True, True]]
print(nan_checker.is_nan(["", None, 'NaN', ["List Value 1", '', None]]))

[True, True, False, [False, True, True]]


In [13]:
# Values can be declared as nan-values
# [True, False, False, True]
nan_checker.is_nan(['NaN', 'Empty', 'None', 'N/A'], nan_vals=['NaN', 'N/A'])

[True, False, False, True]

## `Pattern`
Functionality testing of `Pattern` class. The `Pattern` class stores different patterns and data summaries regarding NaN values. 

In [14]:
reload(impyte)
pattern_log = impyte.Pattern()

#### `Pattern._check_complete_row(row)`
Determines whether a row consists out of only 1s. Method for NaN summary creation

In [15]:
tsc = TestingSetCreator(random_seed=23)

In [16]:
df = tsc.test_set(complete=2, spat1=4, spat2=5)
imp = impyte.Impyter(df)
imp.pattern()

,0,1,2,Count
0,1,NaN,1,5
1,NaN,1,1,4
2,1,1,1,2


In [17]:
complete_contestants = imp.pattern().apply(pattern_log._check_complete_row, axis=1)

In [18]:
complete_contestants.values

array([-1, -1,  2])

The pattern number containing all complete data points can be found by searching for the maximum value in the array.

In [19]:
complete_contestants.max()

2

#### `Pattern._compute_pattern(data, nan_values="", verbose=False)`
Function that checks for missing values and prints out a quick table of a summary of missing values. Includes pattern overview and counts of missing values by column. Computes pattern and returns its pattern structure plus the count of data points for each of these patterns. To get a nice digestible table add `["table"]` selector to output.

In [20]:
pattern_dict = pattern_log._compute_pattern(data)
indices, table = pattern_dict["indices"], pattern_dict["table"]

In [21]:
table

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,NaN,1,1,1,1,1,1,1,1,1,374
2,1,NaN,1,1,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,NaN,1,1,1,1,1,1,374
5,1,1,1,1,NaN,1,1,1,1,1,374
6,1,1,1,1,1,NaN,1,1,1,1,374
7,1,1,1,1,1,1,NaN,1,1,1,374
8,1,1,1,1,1,1,1,NaN,1,1,374
9,1,1,1,1,1,1,1,1,NaN,1,374


In [22]:
indices[0][:10] # first 10 indices of pattern 0

[374, 749, 1124, 1499, 1874, 2249, 2624, 2999, 3374, 3749]

In [23]:
pattern_nr = 0
print("Pattern {} has {:,} rows.".format(pattern_nr, len(indices[0])))

Pattern 0 has 11,259 rows.


#### `Pattern._is_discrete(tmpdata, unique_instances)`
Determines based on dtype and by counting unique instances whether a column contains categorical/discrete or continuous values.

In [24]:
continuous_df = pd.DataFrame([.53, .22, .1, .11, .4, .7])

In [25]:
discrete_df = pd.DataFrame(["all", "your", "base", "are", "belong", "to", "us"])

In [26]:
pattern_log._is_discrete(continuous_df[0], unique_instances=5) # False

False

In [27]:
pattern_log._is_discrete(discrete_df[0], unique_instances=5) # True

True

#### `Pattern._get_discrete_and_continuous(tmpdata, unique_instances)`
Returns the column names of discrete and continuous variables. Column names are stored in lists for easy selection. Those lists are stored in one dictionary object. All continuous column names can be accessed through `['continuous']` all discrete variables through `['discrete']`.

In [28]:
pattern_log._get_discrete_and_continuous(data, unique_instances=5)

{'continuous': ['satisfaction_level',
  'last_evaluation',
  'number_project',
  'average_montly_hours',
  'time_spend_company'],
 'discrete': ['Work_accident',
  'left',
  'promotion_last_5years',
  'sales',
  'salary']}

#### `Pattern._get_unique_vals(data)`
For each column, this method returns an unique value count.

In [29]:
# [93, 66, 7, 216, 9, 3, 3, 3, 11, 4]
pattern_log._get_unique_vals(data)

[93, 66, 7, 216, 9, 3, 3, 3, 11, 4]

In [30]:
df = tsc.test_set(complete=5)
df

[[1, 1, 0.9248652516259452],
 [1, 1, 0.9486057779931771],
 [1, 1, 0.8924333440485793],
 [1, 1, 0.08355067683068362],
 [1, 1, 0.5920272268857353]]

In [31]:
# [1, 1, 5]
pattern_log._get_unique_vals(pd.DataFrame(df))

[1, 1, 5]

#### `Pattern._store_tuple(tup, row_idx, tmp_col_names)`
Internal storage method to save patterns in pattern_log.

#### `Pattern.get_complete_id()`
Returns pattern number of complete data points.

In [32]:
pattern_log.get_complete_id()

0

#### `Pattern.get_column_name(pattern_no)`
Returns column name(s) of nan-pattern.

In [33]:
pattern_log.get_column_name(1)

['satisfaction_level']

#### `Pattern.get_missing_value_percentage(self, data, importance_filter=False)`
Shows missing value percentage and count of unique values in category based on result and actual data table.

In [34]:
pattern_log.get_missing_value_percentage(data)

,Complete,Missing,Percentage,Unique
satisfaction_level,14625,374,2.49 %,93
last_evaluation,14625,374,2.49 %,66
number_project,14625,374,2.49 %,7
average_montly_hours,14625,374,2.49 %,216
time_spend_company,14625,374,2.49 %,9
Work_accident,14625,374,2.49 %,3
left,14625,374,2.49 %,3
promotion_last_5years,14625,374,2.49 %,3
sales,14625,374,2.49 %,11
salary,14625,374,2.49 %,4


#### `Pattern.get_pattern(data=None, unique_instances=10, recompute=False)`
Returns NaN-patterns based on primary computation or initiates new computation of NaN-patterns. Uses `_compute_patterns()` to visualize patterns if not yet computed. Otherwise returns stored information regarding patterns. If a pattern is already computed, the `recompute` flag has to be set to `True` in order to compute a new pattern structure.

In [35]:
reload(impyte)

<module 'impyte' from '../impyte.py'>

In [36]:
pattern_log = impyte.Pattern()

In [37]:
pattern_log.get_pattern(data)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,NaN,1,1,1,1,1,1,1,1,1,374
2,1,NaN,1,1,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,NaN,1,1,1,1,1,1,374
5,1,1,1,1,NaN,1,1,1,1,1,374
6,1,1,1,1,1,NaN,1,1,1,1,374
7,1,1,1,1,1,1,NaN,1,1,1,374
8,1,1,1,1,1,1,1,NaN,1,1,374
9,1,1,1,1,1,1,1,1,NaN,1,374


In [38]:
df_max = tsc.test_set(complete=10000, spat1=2000, spat2=1000, spat3=500, mpat1=300, mpat2=200, mpat3=100)

In [39]:
pattern_log.get_pattern(pd.DataFrame(df_max), recompute=True)

,0,1,2,Count
0,1,1,1,10000
1,NaN,1,1,2000
2,1,NaN,1,1000
3,1,1,NaN,500
4,NaN,NaN,1,300
5,NaN,1,NaN,200
6,1,NaN,NaN,100


#### `Pattern.get_single_nan_pattern_nos()`
Returns all pattern numbers of single nans.

In [40]:
pattern_log.get_single_nan_pattern_nos()

Int64Index([1, 2, 3], dtype='int64')

#### `Pattern.get_multi_nan_pattern_nos(multi=True)`
Returns all pattern numbers of multi-nans or single-nans.

In [41]:
pattern_log.get_multi_nan_pattern_nos()

Int64Index([4, 5, 6], dtype='int64')

#### `Pattern.get_pattern_indices(pattern_no)`

In [42]:
# [374, 749, 1124, 1499, 1874, 2249, 2624, 2999, 3374, 3749]
pattern_log.get_pattern_indices(0)[:10] # get first 10 indices of pattern 0

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

#### `Pattern.get_continuous()`
Returns list with names of all continuous variables.

In [43]:
pattern_log.get_continuous()

[2]

#### `Pattern.get_discrete()`
Returns list with names of all discrete variables.

In [44]:
pattern_log.get_discrete()

[0, 1]

#### `Pattern.remove_pattern(pattern_no)`
Removes a pattern from storage.

In [45]:
reload(impyte)
pattern_log = impyte.Pattern()
pattern_log.get_pattern(pd.DataFrame(df_max))

,0,1,2,Count
0,1,1,1,10000
1,NaN,1,1,2000
2,1,NaN,1,1000
3,1,1,NaN,500
4,NaN,NaN,1,300
5,NaN,1,NaN,200
6,1,NaN,NaN,100


In [46]:
pattern_log.remove_pattern(6)

In [47]:
pattern_log.get_pattern()

,0,1,2,Count
0,1,1,1,10000
1,NaN,1,1,2000
2,1,NaN,1,1000
3,1,1,NaN,500
4,NaN,NaN,1,300
5,NaN,1,NaN,200


## `Impyter`

In [48]:
reload(impyte)
imp = impyte.Impyter() # instantiate impyte class

### Load data into imputer

#### `Impyter._data_check(data)`
Checks if data is pandas DataFrame and transforms otherwise.

In [49]:
df_max[:5]

[[1, 1, 0.9248652516259452],
 [1, 1, 0.9486057779931771],
 [1, 1, 0.8924333440485793],
 [1, 1, 0.08355067683068362],
 [1, 1, 0.5920272268857353]]

In [50]:
imp._data_check(df_max).head()

,0,1,2
0,1.0,1.0,0.924865
1,1.0,1.0,0.948606
2,1.0,1.0,0.892433
3,1.0,1.0,0.083551
4,1.0,1.0,0.592027


In [51]:
imp._data_check([["a", "d", "g"], ["b", "e", "h"], ["c", "f", "i"]])

,0,1,2
0,a,d,g
1,b,e,h
2,c,f,i


#### `Impyter._get_display_options(cols=True)`
Returns pandas display options for better readability of results. 

In [52]:
imp._get_display_options()

20

In [53]:
imp._get_display_options(cols=False)

60

#### `Impyter._set_display_options(length, cols=True)`
Sets individual display options for pattern results. If `cols` set to `False`, the maximum rows displayed can be set.

In [54]:
imp._set_display_options(25)

In [55]:
imp._set_display_options(65, cols=False)

#### `Impyter.load_data()`
Requires a pandas DataFrame to load. Otherwise, the input is being transformed into a DataFrame. While loading the data is being copied into the object, to stay clear of consistency issues with the original data set.

In [56]:
imp.load_data(data)

As an alternative a DataFrame can be handed over while instantiating the Impyter object.

In [57]:
imp = impyte.Impyter(data)

#### `Impyter.get_data()`
Returns the loaded data for quick reference.

In [58]:
imp.get_data().head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.75,3.0,260.0,2.0,0.0,0.0,0.0,technical,low
1,NaN,0.70,5.0,264.0,2.0,0.0,0.0,0.0,technical,low
2,NaN,0.50,3.0,231.0,3.0,0.0,0.0,0.0,sales,low
3,NaN,0.79,4.0,272.0,3.0,0.0,0.0,0.0,technical,low
4,NaN,0.90,3.0,138.0,3.0,0.0,0.0,0.0,marketing,medium


#### `Impyter.pattern()`
Leverages `Pattern._compute_pattern()` and `Pattern.get_pattern()` methods to compute and return an overview of all existant NaN patterns in the data set. The overview shows a `NaN` in the column where a data point was missing and `1` for all complete slots. On the right hand side is a count variable to indicate how often that pattern was found. The patterns are always sorted by count and it is not given, that pattern 0 is always the pattern with only complete cases.

In [59]:
reload(impyte)
imp = impyte.Impyter(data) # instantiate impyte class
imp.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,NaN,1,1,1,1,1,1,1,1,1,374
2,1,NaN,1,1,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,NaN,1,1,1,1,1,1,374
5,1,1,1,1,NaN,1,1,1,1,1,374
6,1,1,1,1,1,NaN,1,1,1,1,374
7,1,1,1,1,1,1,NaN,1,1,1,374
8,1,1,1,1,1,1,1,NaN,1,1,374
9,1,1,1,1,1,1,1,1,NaN,1,374


#### `Impyter.drop_pattern(pattern_no, inplace=False)`
Drops pattern from data set and returns preliminary result. If `inplace` flag is set to `True`, internal storage of impyte object is being manipulated as well.

In [60]:
imp.drop_pattern(4).head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.75,3.0,260.0,2.0,0.0,0.0,0.0,technical,low
1,NaN,0.70,5.0,264.0,2.0,0.0,0.0,0.0,technical,low
2,NaN,0.50,3.0,231.0,3.0,0.0,0.0,0.0,sales,low
3,NaN,0.79,4.0,272.0,3.0,0.0,0.0,0.0,technical,low
4,NaN,0.90,3.0,138.0,3.0,0.0,0.0,0.0,marketing,medium


In [61]:
imp.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,NaN,1,1,1,1,1,1,1,1,1,374
2,1,NaN,1,1,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,NaN,1,1,1,1,1,1,374
5,1,1,1,1,NaN,1,1,1,1,1,374
6,1,1,1,1,1,NaN,1,1,1,1,374
7,1,1,1,1,1,1,NaN,1,1,1,374
8,1,1,1,1,1,1,1,NaN,1,1,374
9,1,1,1,1,1,1,1,1,NaN,1,374


If inplace flag set to True, the changes happen in the data set that's being stored in the Impyte object. Otherwise, a copy without the dropped pattern will be returned and the stored data set stays intact.

In [62]:
_ = imp.drop_pattern(4, inplace=True)

In [63]:
imp.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,NaN,1,1,1,1,1,1,1,1,1,374
2,1,NaN,1,1,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
5,1,1,1,1,NaN,1,1,1,1,1,374
6,1,1,1,1,1,NaN,1,1,1,1,374
7,1,1,1,1,1,1,NaN,1,1,1,374
8,1,1,1,1,1,1,1,NaN,1,1,374
9,1,1,1,1,1,1,1,1,NaN,1,374
10,1,1,1,1,1,1,1,1,1,NaN,374


#### `Impyter.get_pattern(pattern_no)`
Returns data points for a specific pattern_no for further investigation.

In [64]:
imp.get_pattern(1).head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.75,3.0,260.0,2.0,0.0,0.0,0.0,technical,low
1,NaN,0.70,5.0,264.0,2.0,0.0,0.0,0.0,technical,low
2,NaN,0.50,3.0,231.0,3.0,0.0,0.0,0.0,sales,low
3,NaN,0.79,4.0,272.0,3.0,0.0,0.0,0.0,technical,low
4,NaN,0.90,3.0,138.0,3.0,0.0,0.0,0.0,marketing,medium


#### `Impyter.get_summary()`
Returns table with information on missing values per column, its percentage and the count of unique values within that column.

In [65]:
imp.get_summary()

,Complete,Missing,Percentage,Unique
satisfaction_level,14251,374,2.56 %,93
last_evaluation,14251,374,2.56 %,66
number_project,14251,374,2.56 %,7
time_spend_company,14251,374,2.56 %,9
Work_accident,14251,374,2.56 %,3
left,14251,374,2.56 %,3
promotion_last_5years,14251,374,2.56 %,3
sales,14251,374,2.56 %,11
salary,14251,374,2.56 %,4


In [66]:
data['Work_accident'].unique()

array([  0.,   1.,  nan])

Setting the importance filter flag to `True`  shows only columns that have some missing values. This is helpful for data sets with a large amount of variables and only few nan-values.

In [67]:
for pattern_no in range(1,4): #  drop patterns 1 to 3
    imp.drop_pattern(pattern_no, inplace=True)

In [68]:
imp.get_summary(importance_filter=True)

,Complete,Missing,Percentage,Unique
time_spend_company,13129,374,2.77 %,9
Work_accident,13129,374,2.77 %,3
left,13129,374,2.77 %,3
promotion_last_5years,13129,374,2.77 %,3
sales,13129,374,2.77 %,11
salary,13129,374,2.77 %,4


#### `Impyte.one_hot_encode()`
Relies on `pandas.get_dummies()` method to transform categorical values into one-hot-encoded values.

In [69]:
reload(impyte)
imputer = impyte.Impyter()
imputer.load_data(data)
_ = imputer.pattern()

In [70]:
ohe_data = imputer.one_hot_encode(data)
ohe_data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_ohe_IT,sales_ohe_RandD,sales_ohe_accounting,sales_ohe_hr,sales_ohe_management,sales_ohe_marketing,sales_ohe_product_mng,sales_ohe_sales,sales_ohe_support,sales_ohe_technical,salary_ohe_high,salary_ohe_low,salary_ohe_medium
0,NaN,0.75,3.0,260.0,2.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,1,0,1,0
1,NaN,0.70,5.0,264.0,2.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,1,0,1,0
2,NaN,0.50,3.0,231.0,3.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,1,0
3,NaN,0.79,4.0,272.0,3.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,1,0,1,0
4,NaN,0.90,3.0,138.0,3.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,1


#### `Impyte.one_hot_decode()`
The inversion method to `Impyte.one_hot_encode()`. Transforms one-hot-encoded columns back to categorical values.

In [71]:
imputer.one_hot_decode(ohe_data).head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.75,3.0,260.0,2.0,0.0,0.0,0.0,technical,low
1,NaN,0.70,5.0,264.0,2.0,0.0,0.0,0.0,technical,low
2,NaN,0.50,3.0,231.0,3.0,0.0,0.0,0.0,sales,low
3,NaN,0.79,4.0,272.0,3.0,0.0,0.0,0.0,technical,low
4,NaN,0.90,3.0,138.0,3.0,0.0,0.0,0.0,marketing,medium


#### `Impyter.impute()`
Impute is the core method of impyte. The method works out of the box and uses Random Forest estimators per default to impute missing values. It automatically performs cross-validation to showcase the potential accuracy of the imputation. Scoring that is being used is f1_macro score for classifiers (supporting binary and multi-class) and r2 for regression models.

In [72]:
reload(impyte)
imputer = impyte.Impyter()
imputer.load_data(data)

In [73]:
_ = imputer.pattern()
complete_df = imputer.impute(estimator='rf')

Scoring Threshold             Classification                Regression                    
                              None                          None                          

Pattern: Label                Score                         Estimator                     
1: satisfaction_level         0.472 (r2)                    RandomForestRegressor           imputed...
2: last_evaluation            0.287 (r2)                    RandomForestRegressor           imputed...
3: number_project             0.572 (f1_macro)              RandomForestClassifier          imputed...
4: average_montly_hours       0.324 (r2)                    RandomForestRegressor           imputed...
5: time_spend_company         0.592 (f1_macro)              RandomForestClassifier          imputed...
6: Work_accident              0.601 (f1_macro)              RandomForestClassifier          imputed...
7: left                       0.980 (f1_macro)              RandomForestClassifier          imputed...
8: pr

In [74]:
complete_df = imputer.impute(estimator='rf')

Scoring Threshold             Classification                Regression                    
                              None                          None                          

Pattern: Label                Score                         Estimator                     
1: satisfaction_level         0.458 (r2)                    RandomForestRegressor           imputed...
2: last_evaluation            0.296 (r2)                    RandomForestRegressor           imputed...
3: number_project             0.569 (f1_macro)              RandomForestClassifier          imputed...
4: average_montly_hours       0.318 (r2)                    RandomForestRegressor           imputed...
5: time_spend_company         0.594 (f1_macro)              RandomForestClassifier          imputed...
6: Work_accident              0.598 (f1_macro)              RandomForestClassifier          imputed...
7: left                       0.981 (f1_macro)              RandomForestClassifier          imputed...
8: pr

##### Investigate completed data set

In [75]:
imp = impyte.Impyter(complete_df)

In [76]:
imp.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,14999


##### Scoring measures
In order to fill in only columns that surpass a certain scoring threshold (i.e. f1 score > .7), the threshold parameter can be set. The threshold values are being transmitted through a dictionary.

In [77]:
complete_df_threshold = imputer.impute(estimator='rf', threshold={"f1_macro": .7,
                                                                  "r2": .5})

Scoring Threshold             Classification                Regression                    
                              0.7                           0.5                           

Pattern: Label                Score                         Estimator                     
1: satisfaction_level         0.468 (r2)                    RandomForestRegressor           not imputed...
2: last_evaluation            0.293 (r2)                    RandomForestRegressor           not imputed...
3: number_project             0.574 (f1_macro)              RandomForestClassifier          not imputed...
4: average_montly_hours       0.334 (r2)                    RandomForestRegressor           not imputed...
5: time_spend_company         0.593 (f1_macro)              RandomForestClassifier          not imputed...
6: Work_accident              0.603 (f1_macro)              RandomForestClassifier          not imputed...
7: left                       0.980 (f1_macro)              RandomForestClassifier  

In [78]:
len(imputer.get_result())

14999

In [79]:
imputer.get_pattern(7, result=True)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
2250,0.24,0.74,5.0,282.0,3.0,0.0,1.0,0.0,IT,medium
2251,0.49,0.37,4.0,216.0,4.0,0.0,0.0,0.0,technical,low
2252,0.96,0.71,4.0,210.0,3.0,0.0,1.0,0.0,technical,medium
2253,0.18,0.76,5.0,173.0,4.0,0.0,1.0,0.0,support,medium
2254,0.80,0.69,3.0,164.0,3.0,0.0,1.0,0.0,support,medium
2255,0.43,0.71,3.0,186.0,2.0,0.0,1.0,0.0,accounting,medium
2256,0.95,0.71,3.0,223.0,2.0,0.0,1.0,0.0,sales,low
2257,0.72,0.72,4.0,218.0,4.0,0.0,1.0,0.0,IT,low
2258,0.76,0.79,3.0,161.0,3.0,0.0,0.0,0.0,technical,low
2259,0.86,0.93,5.0,223.0,5.0,0.0,1.0,0.0,RandD,medium


In [80]:
imp = impyte.Impyter(complete_df_threshold)
imp.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,12007
1,NaN,1,1,1,1,1,1,1,1,1,374
2,1,NaN,1,1,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,NaN,1,1,1,1,1,1,374
5,1,1,1,1,NaN,1,1,1,1,1,374
6,1,1,1,1,1,NaN,1,1,1,1,374
7,1,1,1,1,1,1,1,1,NaN,1,374
8,1,1,1,1,1,1,1,1,1,NaN,374


##### Multi-Nans
Prediction of values with multi-nan is a last resort option. This might be suitable for certain edge cases but if the score values are low it should be considered dropping the feature or the data points.

In [81]:
reload(impyte)
multi_data = data.copy()
import numpy as np
for i in range(0, 100):
    multi_data.at[i, "last_evaluation"] = np.nan
reload(impyte)
imp = impyte.Impyter(multi_data)
_ = imp.pattern()
res = imp.impute(estimator="rf", multi_nans=True)

Scoring Threshold             Classification                Regression                    
                              None                          None                          

Pattern: Label                Score                         Estimator                     
1: last_evaluation            0.292 (r2)                    RandomForestRegressor           imputed...
2: number_project             0.575 (f1_macro)              RandomForestClassifier          imputed...
3: average_montly_hours       0.326 (r2)                    RandomForestRegressor           imputed...
4: time_spend_company         0.593 (f1_macro)              RandomForestClassifier          imputed...
5: Work_accident              0.599 (f1_macro)              RandomForestClassifier          imputed...
6: left                       0.983 (f1_macro)              RandomForestClassifier          imputed...
7: promotion_last_5years      0.760 (f1_macro)              RandomForestClassifier          imputed...
8: sa

In [82]:
res = imp.impute(estimator="rf", multi_nans=True)

Scoring Threshold             Classification                Regression                    
                              None                          None                          

Pattern: Label                Score                         Estimator                     
1: last_evaluation            0.296 (r2)                    RandomForestRegressor           imputed...
2: number_project             0.572 (f1_macro)              RandomForestClassifier          imputed...
3: average_montly_hours       0.323 (r2)                    RandomForestRegressor           imputed...
4: time_spend_company         0.596 (f1_macro)              RandomForestClassifier          imputed...
5: Work_accident              0.596 (f1_macro)              RandomForestClassifier          imputed...
6: left                       0.980 (f1_macro)              RandomForestClassifier          imputed...
7: promotion_last_5years      0.737 (f1_macro)              RandomForestClassifier          imputed...
8: sa

In [83]:
imp.save_model(filename='all_models_hr.pkl')

In [84]:
reload(impyte)

<module 'impyte' from '../impyte.py'>

In [85]:
imp2 = impyte.Impyter()

In [86]:
imp2.load_data(multi_data)

In [87]:
imp2.load_model('all_models_hr.pkl')

Computing NaN-patterns first ...

Found 11 models...
Added model for pattern 1
Added model for pattern 2
Added model for pattern 3
Added model for pattern 4
Added model for pattern 5
Added model for pattern 6
Added model for pattern 7
Added model for pattern 8
Added model for pattern 9
Added model for pattern 10
Added model for pattern 11


#### `Impyter.drop_imputation()`
In the case of multi-nan, `drop_imputation` will average the score of all models. Yet, performing this method for multi-nan patterns is discouraged. Further individual treatment of the data set might be more helpful in order to preprocess the information correctly. One potential action could be, to drop multi-nan columns if they contain no information.

In [88]:
imp2.drop_imputation({"f1_macro": .5,
                      "r2": .3})

Dropping pattern 1 (0.2957929020137531 < 0.3 r2)
Dropping pattern 8 (0.33362372360150816 < 0.5 f1_macro)
Dropping pattern 11 (0.23830913044971552 < 0.3 r2)


In [89]:
imp2.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,NaN,1,1,1,1,1,1,1,1,374
2,1,1,NaN,1,1,1,1,1,1,1,374
3,1,1,1,NaN,1,1,1,1,1,1,374
4,1,1,1,1,NaN,1,1,1,1,1,374
5,1,1,1,1,1,NaN,1,1,1,1,374
6,1,1,1,1,1,1,NaN,1,1,1,374
7,1,1,1,1,1,1,1,NaN,1,1,374
8,1,1,1,1,1,1,1,1,NaN,1,374
9,1,1,1,1,1,1,1,1,1,NaN,374


Drop imputation can also be used to drop the pattern, if imputation results aren't sufficient enough. If `drop_pattern` is set to `True`, impyte will automatically remove each pattern that doesn't reach the threshold.

In [90]:
imp2.load_data(multi_data)

In [91]:
imp2.load_model('all_models_hr.pkl')

Computing NaN-patterns first ...

Found 11 models...
Added model for pattern 1
Added model for pattern 2
Added model for pattern 3
Added model for pattern 4
Added model for pattern 5
Added model for pattern 6
Added model for pattern 7
Added model for pattern 8
Added model for pattern 9
Added model for pattern 10
Added model for pattern 11


In [92]:
imp2.drop_imputation({"f1_macro": .5,
                      "r2": .3}, drop_pattern=True)

Dropping pattern 1 (0.2957929020137531 < 0.3 r2)
Dropping pattern 8 (0.33362372360150816 < 0.5 f1_macro)
Dropping pattern 11 (0.23830913044971552 < 0.3 r2)


In [93]:
imp2.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
2,1,1,NaN,1,1,1,1,1,1,1,374
3,1,1,1,NaN,1,1,1,1,1,1,374
4,1,1,1,1,NaN,1,1,1,1,1,374
5,1,1,1,1,1,NaN,1,1,1,1,374
6,1,1,1,1,1,1,NaN,1,1,1,374
7,1,1,1,1,1,1,1,NaN,1,1,374
9,1,1,1,1,1,1,1,1,1,NaN,374
10,NaN,1,1,1,1,1,1,1,1,1,274


#### `Impyter.get_result()`
Returns the results data set, once impute was performed. Before imputation this method returns the original data set.

In [94]:
imp.get_result().head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.504,0.642,3.0,260.0,2.0,0.0,0.0,0.0,technical,low
1,0.504,0.642,5.0,264.0,2.0,0.0,0.0,0.0,technical,low
2,0.247,0.706,3.0,231.0,3.0,0.0,0.0,0.0,sales,low
3,0.247,0.642,4.0,272.0,3.0,0.0,0.0,0.0,technical,low
4,0.230,0.682,3.0,138.0,3.0,0.0,0.0,0.0,marketing,medium


In [95]:
len(imp.get_result())

14999

#### `Impyter.get_model()`

In [96]:
model = imp.get_model(2)

In [97]:
model.feature_name[0]

'number_project'

In [98]:
model.estimator_name[0]

'RandomForestClassifier'

In [99]:
model.scoring[0]

'f1_macro'

In [100]:
model.score

[0.57202860256980426]

In [101]:
model.model[0]

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [102]:
reload(impyte)
imp_max = impyte.Impyter(df_max)

In [103]:
imp_max.pattern()

,0,1,2,Count
0,1,1,1,10000
1,NaN,1,1,2000
2,1,NaN,1,1000
3,1,1,NaN,500
4,NaN,NaN,1,300
5,NaN,1,NaN,200
6,1,NaN,NaN,100


In [104]:
imp_max.pattern_log.pattern_dependent_dict

{('NaN', 'NaN', 1): [0, 1],
 ('NaN', 1, 'NaN'): [0, 2],
 ('NaN', 1, 1): [0],
 (1, 'NaN', 'NaN'): [1, 2],
 (1, 'NaN', 1): [1],
 (1, 1, 'NaN'): [2],
 (1, 1, 1): []}

In [105]:
_ = imp_max.impute(multi_nans=True)

Scoring Threshold             Classification                Regression                    
                              None                          None                          

Pattern: Label                Score                         Estimator                     
1: 0                          1.000 (f1_macro)              RandomForestClassifier          imputed...
2: 1                          1.000 (f1_macro)              RandomForestClassifier          imputed...
3: 2                          -0.000 (r2)                   RandomForestRegressor           imputed...

Multi nans
4: 0                          1.000 (f1_macro)              RandomForestClassifier          imputed...
4: 1                          1.000 (f1_macro)              RandomForestClassifier          imputed...
5: 0                          1.000 (f1_macro)              RandomForestClassifier          imputed...
5: 2                          -0.000 (r2)                   RandomForestRegressor           impu

In [106]:
mmdl = imp_max.model_log[4]

In [107]:
mmdl.model_list

[<impyte.ImpyterModel at 0x1a1833e908>, <impyte.ImpyterModel at 0x1a1a556ef0>]

In [108]:
mmdl.get_dependend_and_independent_variables()

{'dependent_variables': [0, 1], 'independent_variables': [2]}

#### `Impyter.save_model(pattern_no=None, name=None)`
Stores an imputation model for either the whole data set or a particular pattern in a pickle file. If `pattern_no` is not set, the method stores all models. If `name` is not set, an automated name is being produced including a timestamp.

In [109]:
imp_max.save_model(filename='all_models_dummy.pkl')

In [110]:
imp_max.save_model(2, filename='one_model_dummy.pkl')

#### `Impyter.map_model_to_pattern(mdl)`
Checks model for similarity to stored patterns and returns pattern number if a match is found.

In [111]:
mdl = imp_max.get_model(2)

In [112]:
print(imp_max.map_model_to_pattern(mdl))

2


#### `Impyter.map_model_to_pattern(mdl)`
Checks model for similarity to stored patterns and returns pattern number if a match is found.

In [113]:
mdl = imp_max.get_model(4)

In [114]:
print(imp_max.map_multimodel_to_pattern(mdl))

4


#### `Impyter.load_model(model)`
Load a stored machine learning model to perform value imputation. If model is a list of models, all models will be checked according to their independent and dependent variables. If a matching pattern is found, the model will be updated.

In [115]:
# 2017-12-01_impyte_mdl_1512153316.pkl
imp_max.load_model(filename='all_models_dummy.pkl')

Found 6 models...
Added model for pattern 1
Added model for pattern 2
Added model for pattern 3
Added model for pattern 4
Added model for pattern 5
Added model for pattern 6


In [116]:
imp_max.load_model(filename='one_model_dummy.pkl')

Added model for pattern 2


#### `Impyter.compare_features(list1, list2)`
Compares two lists given its objects based on a comparison of Counter dicts. The order of all elements is unimportant.

In [117]:
# [True]
imp.compare_features(["one", "two", "three"], ["one", "two", "three"])

True

In [118]:
# [True]
imp.compare_features(["one", "two", "three"], ["one", "three", "two"])

True

In [119]:
# [False]
imp.compare_features(["one", "two", "three"], ["one", "two", "four"])

False

### Timing some of the functions
Below is a piece of information on efficiency of the functions. This doesn't reflect complexity of the functions, but it will show an runtime estimate for your local machine.

In [120]:
%timeit nan_checker.is_nan(["", 'None', 'NaN'])
%timeit nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

3.86 µs ± 227 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
4.06 µs ± 421 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Use Cases

### 1. Visualization of missing information
`Impyte` has two essential ways of showing the different types of missing information in a data structure.

#### I. Show patterns
`Impyte` shows nan-patterns in the data set and presents them in tabular form.

In [122]:
reload(impyte)
from tools.testing_sets import TestingSetCreator
tsc = TestingSetCreator(random_seed=23)

In [123]:
min_df = pd.DataFrame(tsc.test_set(complete=2, spat1=2, spat2=2, spat3=2))

In [124]:
min_df

,0,1,2
0,1.0,1.0,0.924865
1,1.0,1.0,0.948606
2,NaN,1.0,0.892433
3,NaN,1.0,0.083551
4,1.0,NaN,0.592027
5,1.0,NaN,0.423747
6,1.0,1.0,NaN
7,1.0,1.0,NaN


In [125]:
# user imports `impyte` and instantiates an Impyter object
reload(impyte)
import impyte
imp = impyte.Impyter()

In [126]:
# data the user wants to analyze is being loaded
imp.load_data(min_df)

In [127]:
imp.set_unique(3)  # otherwise impyte will categorize 

In [128]:
# user calls Impyte.pattern()
imp.pattern()

,0,1,2,Count
0,1,1,1,2
1,NaN,1,1,2
2,1,NaN,1,2
3,1,1,NaN,2


In [129]:
# user can select a certain pattern by calling Impyte.get_pattern(index_no)
# i.e. to get pattern 1
imp.get_pattern(1)

,0,1,2
2,NaN,1.0,0.892433
3,NaN,1.0,0.083551


In [130]:
# user can drop a certain pattern by calling Impyte.drop_pattern(index_no)
# i.e. drop pattern 1
imp.drop_pattern(1)

,0,1,2
0,1.0,1.0,0.924865
1,1.0,1.0,0.948606
4,1.0,NaN,0.592027
5,1.0,NaN,0.423747
6,1.0,1.0,NaN
7,1.0,1.0,NaN


In [131]:
# by setting the inplace parameter to True, the result data set can be augmented 
imp.drop_pattern(1, inplace=True)

,0,1,2
0,1.0,1.0,0.924865
1,1.0,1.0,0.948606
4,1.0,NaN,0.592027
5,1.0,NaN,0.423747
6,1.0,1.0,NaN
7,1.0,1.0,NaN


In [132]:
imp.get_result()

,0,1,2
0,1.0,1.0,0.924865
1,1.0,1.0,0.948606
4,1.0,NaN,0.592027
5,1.0,NaN,0.423747
6,1.0,1.0,NaN
7,1.0,1.0,NaN


In [133]:
# The original data set stays in tact
imp.get_data()

,0,1,2
0,1.0,1.0,0.924865
1,1.0,1.0,0.948606
2,NaN,1.0,0.892433
3,NaN,1.0,0.083551
4,1.0,NaN,0.592027
5,1.0,NaN,0.423747
6,1.0,1.0,NaN
7,1.0,1.0,NaN


In [134]:
# The pattern log is tied to the results data set
imp.pattern()

,0,1,2,Count
0,1,1,1,2
2,1,NaN,1,2
3,1,1,NaN,2


In [135]:
# To recompute the nan-patterns based on the original data
# simply set the recompute flag to True
imp.pattern(recompute=True)

,0,1,2,Count
0,1,1,1,2
1,NaN,1,1,2
2,1,NaN,1,2
3,1,1,NaN,2


#### II. Data Summarization
The second visualization method analyzes the data on a value level. By calling `Impyte.get_summary` a description of all variables is being returned containing the amount of complete data points, missing data points, the relative value in respect of all data points and an indicator that shows how many unique values are being presented.

> Note: NaN values are being counted as unique values as well.

In [136]:
reload(impyte)

<module 'impyte' from '../impyte.py'>

In [137]:
imp = impyte.Impyter(min_df)

In [138]:
print(imp.pattern())

     0    1    2  Count
0    1    1    1      2
1  NaN    1    1      2
2    1  NaN    1      2
3    1    1  NaN      2


In [139]:
imp.get_summary(importance_filter=False)

,Complete,Missing,Percentage,Unique
0,6,2,25.00 %,2
1,6,2,25.00 %,2
2,6,2,25.00 %,7


In [140]:
imp.drop_pattern(1, inplace=True)

,0,1,2
0,1.0,1.0,0.924865
1,1.0,1.0,0.948606
4,1.0,NaN,0.592027
5,1.0,NaN,0.423747
6,1.0,1.0,NaN
7,1.0,1.0,NaN


In [141]:
imp.get_summary(importance_filter=False)

,Complete,Missing,Percentage,Unique
0,6,0,0.00 %,1
1,4,2,33.33 %,2
2,4,2,33.33 %,5


In [142]:
imp.pattern(recompute=True)

,0,1,2,Count
0,1,1,1,2
1,NaN,1,1,2
2,1,NaN,1,2
3,1,1,NaN,2


In [143]:
imp.get_summary(importance_filter=False)

,Complete,Missing,Percentage,Unique
0,6,2,25.00 %,2
1,6,2,25.00 %,2
2,6,2,25.00 %,7


### 2. Missing Value Imputation

In [144]:
df_med = pd.DataFrame(tsc.test_set(complete=1000, spat1=500, spat2=400, spat3=300))

In [145]:
# user imports impyte and instantiates Impyter object
import impyte
reload(impyte)
imp = impyte.Impyter()

In [146]:
# user loads dataset
imp.load_data(df_med)

In [147]:
# user calls impute method
_ = imp.impute()

Computing NaN-patterns first ...

Scoring Threshold             Classification                Regression                    
                              None                          None                          

Pattern: Label                Score                         Estimator                     
1: 0                          1.000 (f1_macro)              RandomForestClassifier          imputed...
2: 1                          1.000 (f1_macro)              RandomForestClassifier          imputed...
3: 2                          -0.008 (r2)                   RandomForestRegressor           imputed...


### 3. Saving the models

In [148]:
imp.save_model(filename='test.pkl')

In [149]:
imp.load_data(df_med)

In [150]:
imp.load_model(filename='test.pkl')

Computing NaN-patterns first ...

Found 3 models...
Added model for pattern 1
Added model for pattern 2
Added model for pattern 3


In [151]:
imp.pattern_log.result_pattern

{('NaN', 1, 1): 500, (1, 'NaN', 1): 400, (1, 1, 'NaN'): 300, (1, 1, 1): 1000}

In [152]:
imputer.pattern_log.get_single_nan_pattern_nos()

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype='int64')

In [153]:
reload(impyte)

<module 'impyte' from '../impyte.py'>

In [154]:
pat = impyte.Pattern()

In [155]:
print(pat)

<Pattern: Empty Pattern Object>


## 1-Dimensional Arrays

In [156]:
reload(impyte)
imp_1d = impyte.Impyter()
df_1d = pd.DataFrame([1, 2, 3, 4, 5])
df_1d

,0
0,1
1,2
2,3
3,4
4,5


In [157]:
#imp_1d.load_data(df_1d)

In [158]:
#df_1d = df_1d.transpose()

In [159]:
df_1d

,0
0,1
1,2
2,3
3,4
4,5


In [160]:
#imp_1d.load_data(df_1d)

In [161]:
#imp_1d.impute()

In [162]:
imp_1d.load_data(df)

In [163]:
df

[[1, 1, 0.9248652516259452],
 [1, 1, 0.9486057779931771],
 [1, 1, 0.8924333440485793],
 [1, 1, 0.08355067683068362],
 [1, 1, 0.5920272268857353]]

In [164]:
reload(impyte)
imp_1d = impyte.Impyter()
df_1d = pd.DataFrame([1, 2, 3, 4, 5])
imp_1d.load_data(data)

In [167]:
imp_1d.impute(estimator='rf')

Scoring Threshold             Classification                Regression                    
                              None                          None                          

Pattern: Label                Score                         Estimator                     
1: satisfaction_level         0.467 (r2)                    RandomForestRegressor           imputed...
2: last_evaluation            0.288 (r2)                    RandomForestRegressor           imputed...
3: number_project             0.571 (f1_macro)              RandomForestClassifier          imputed...
4: average_montly_hours       0.316 (r2)                    RandomForestRegressor           imputed...
5: time_spend_company         0.595 (f1_macro)              RandomForestClassifier          imputed...
6: Work_accident              0.598 (f1_macro)              RandomForestClassifier          imputed...
7: left                       0.981 (f1_macro)              RandomForestClassifier          imputed...
8: pr

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.422,0.75,3.0,260.0,2.0,0.0,0.0,0.0,technical,low
1,0.422,0.70,5.0,264.0,2.0,0.0,0.0,0.0,technical,low
2,0.361,0.50,3.0,231.0,3.0,0.0,0.0,0.0,sales,low
3,0.422,0.79,4.0,272.0,3.0,0.0,0.0,0.0,technical,low
4,0.397,0.90,3.0,138.0,3.0,0.0,0.0,0.0,marketing,medium
5,0.222,0.60,3.0,149.0,4.0,0.0,0.0,0.0,sales,medium
6,0.760,0.97,6.0,284.0,4.0,0.0,1.0,0.0,sales,low
7,0.760,0.85,5.0,246.0,5.0,0.0,1.0,0.0,sales,medium
8,0.760,0.92,4.0,246.0,5.0,0.0,1.0,0.0,sales,low
9,0.760,0.46,2.0,153.0,3.0,1.0,1.0,0.0,technical,medium


In [168]:
imp_1d.impute(estimator='sgd')

Scoring Threshold             Classification                Regression                    
                              None                          None                          

Pattern: Label                Score                         Estimator                     
1: satisfaction_level         0.189 (r2)                    SGDRegressor                    imputed...
2: last_evaluation            0.186 (r2)                    SGDRegressor                    imputed...
3: number_project             0.369 (f1_macro)              SGDClassifier                   imputed...
4: average_montly_hours       0.213 (r2)                    SGDRegressor                    imputed...
5: time_spend_company (* 1)   0.000 (f1_macro)              SGDClassifier                   imputed...
6: Work_accident (* 2)        0.000 (f1_macro)              SGDClassifier                   imputed...
7: left                       0.639 (f1_macro)              SGDClassifier                   imputed...
8: pr

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,4.226915,0.75,3.0,260.0,2.0,0.0,0.0,0.0,technical,low
1,4.188613,0.70,5.0,264.0,2.0,0.0,0.0,0.0,technical,low
2,3.794112,0.50,3.0,231.0,3.0,0.0,0.0,0.0,sales,low
3,4.348479,0.79,4.0,272.0,3.0,0.0,0.0,0.0,technical,low
4,2.481801,0.90,3.0,138.0,3.0,0.0,0.0,0.0,marketing,medium
5,2.622594,0.60,3.0,149.0,4.0,0.0,0.0,0.0,sales,medium
6,4.329498,0.97,6.0,284.0,4.0,0.0,1.0,0.0,sales,low
7,3.826741,0.85,5.0,246.0,5.0,0.0,1.0,0.0,sales,medium
8,3.870740,0.92,4.0,246.0,5.0,0.0,1.0,0.0,sales,low
9,2.633663,0.46,2.0,153.0,3.0,1.0,1.0,0.0,technical,medium
